In [ ]:
%load_ext autoreload
%autoreload 2

Use the HDF5 file generated in notebook 28 to train an XGBoost model.

In [ ]:
import h5py
import numpy as np
import os
import pathlib
import xgboost as xgb

In [ ]:
DATA_DIR = pathlib.Path(os.getenv('DATA_DIR'))
INPUT_FILE = DATA_DIR / '2021-03-17-ppdataset/test.hdf'

In [ ]:
dataset = h5py.File(INPUT_FILE, 'r')

In [ ]:
dataset_x = dataset['/train/x']

In [ ]:
dataset_x.shape

In [ ]:
dataset_y = dataset['/train/y']

In [ ]:
dataset_stations = dataset['/train/stations']

In [ ]:
one_hot = np.zeros((100000, 2359), dtype=bool)
one_hot[np.arange(100000),dataset_stations[:100000]] = True

In [ ]:
one_hot.sum(axis=1)

In [ ]:
in_features = np.concatenate((one_hot, dataset_x[:100000]), axis=1)

In [ ]:
one_hot.shape

In [ ]:
in_features.shape

In [ ]:
feature_names = [f'stn{i}' for i in range(2359)]
feature_names.extend([
 'year',
    'hour',
    'step',
 'latitude',
 'longitude',
 'elevation',
 'gdps_prate',
 'gdps_prmsl',
 'gdps_2t',
 'gdps_2d',
 'gdps_2r',
 'gdps_10u',
 'gdps_10v',
 'gdps_10si',
 'gdps_10wdir',
 'gdps_al',
 'gdps_t_850',
 'gdps_t_500',
 'gdps_gh_1000',
 'gdps_gh_850',
 'gdps_gh_500',
 'gdps_u_500',
 'gdps_v_500',
 'gdps_q_850',
 'gdps_q_500',
 'gdps_thick'])

In [ ]:
dtrain = xgb.DMatrix(in_features, label=dataset_y[:100000], feature_names=feature_names)

In [ ]:
param = {'max_depth':4, 'eta':1, 'objective':'reg:squarederror', 'eval_metric': ['rmse', 'mae'] }

In [ ]:
booster = xgb.train(param, dtrain)

In [ ]:
booster.eval(dtrain)

In [ ]:
val_x = dataset['/val/x'][:100000]

In [ ]:
val_y = dataset['/val/y'][:100000]

In [ ]:
val_one_hot = np.zeros((100000, 2359), dtype=bool)
val_one_hot[np.arange(100000),dataset['/val/stations'][:100000]] = True

In [ ]:
val_x = np.concatenate((val_one_hot, val_x), axis=1)

In [ ]:
len(feature_names)

In [ ]:
val_x.shape

In [ ]:
dval = xgb.DMatrix(val_x, label=val_y, feature_names=feature_names)

In [ ]:
booster.eval(dval)

In [ ]:
xgb.plot_importance(booster)

In [ ]:
booster.get_score(importance_type='weight')

In [ ]:
xgb.plot_tree(booster)